<a href="https://colab.research.google.com/github/Gautam-Chauhan/MAPS-Internship-Code/blob/main/Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object Detection Notebook

In this notebook, we will fine-tune a pre-trained model for object detection with the TensorFlow Object Detection API.

This notebook will closely follow the official TensorFlow Object Detection Tutorial: https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#

We will store the datasets we need in a google drive. So firstly, allow access to my google drive:

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Get access to the object detection API from a GitHub repository:

In [2]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir("..")
elif not pathlib.Path("models").exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [3]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for object_detection: filename=object_detection-0.1-py3-none-any.whl size=1697325 sha256=d4247dcaab4d3bff17fb3088ae3b4be4a1910bbaa876b3a12423a086b3e5a6ff
  Stored in directory: /tmp/pip-ephem-wheel-cache-x6i990y3/wheels/95/4a/63/b2d36ca06eab841de19a38993ffaf2beac152a44539bc642a6
Successfully built object_detection
  Attempting uninstall: object_detection
    Found existing installation: object_detection 0.1
    Uninstalling object_detection-0.1:
      Successfully uninstalled object_detection-0.1


In [4]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

Run the model builder test to check whether installation succeeded:

In [5]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

2025-07-15 18:07:21.494572: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752602841.520523   13491 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752602841.527610   13491 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752602841.546157   13491 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752602841.546220   13491 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752602841.546225   13491 computation_placer.cc:177] computation placer alr